In [48]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Symboles de Futures contracts sur yfinance
##### -----------------------------------------
##### S&P 500 E-Mini Futures : "ES=F"
##### Nasdaq-100 E-Mini Futures : "NQ=F"
##### Pétrole Brut (WTI) : "CL=F"
##### Or : "GC=F"
##### Argent : "SI=F"

### Historical Data

In [8]:
yf.download('AAPL', start='2024-01-01')['Adj Close'].shift(1)

[*********************100%%**********************]  1 of 1 completed


Date
2024-01-02           NaN
2024-01-03    184.938217
2024-01-04    183.553467
2024-01-05    181.222336
2024-01-08    180.495071
                 ...    
2024-10-21    235.000000
2024-10-22    236.479996
2024-10-23    235.860001
2024-10-24    230.759995
2024-10-25    230.570007
Name: Adj Close, Length: 207, dtype: float64

In [18]:
share=yf.download('AAPL', start='2024-01-01')
share.dropna(inplace=True)
share.ffill(inplace=True)
share['Log Return']=np.log(share['Adj Close']/share['Adj Close'].shift(1))
share_var=share['Log Return'].std()*np.sqrt(252)
share_var

[*********************100%%**********************]  1 of 1 completed


0.23478046044097464

In [19]:
futures=yf.download("ES=F", start="2024-01-01")
futures.dropna(inplace=True)
futures.ffill(inplace=True)
futures['Log Return']=np.log(futures['Adj Close']/futures['Adj Close'].shift(1))
futures_var=futures['Log Return'].std()*np.sqrt(252)
futures_var

[*********************100%%**********************]  1 of 1 completed


0.1268509214830126

In [20]:
rho=share['Log Return'].corr(futures['Log Return'])
rho

0.5385873278102166

In [23]:
h = rho*(share_var/futures_var)
h

0.9968376999759554

In [24]:
### VA : amount in asset A
### VF : amount in futures contracts
### Number of contracts to hedge

VA=100
VF=120

N=h*(VA/VF)
N

0.8306980833132962

In [30]:
def cross_hedge (share_ticker, futures_ticker, QA, QF, start_date = "2024-01-01"):
    """
    share_ticker : ticker of share used to hedge the futures
    futures_ticker : ticker of the futures to hedge
    QA : size of position being hedged
    QF : size of futures contract
    """
    ### Compute share variance
    share=yf.download(share_ticker, start=start_date)
    share.dropna(inplace=True)
    share.ffill(inplace=True)
    share['Log Return']=np.log(share['Adj Close']/share['Adj Close'].shift(1))
    share_var=share['Log Return'].std()*np.sqrt(252)
    share_price = share['Adj Close'].iloc[-1]

    ### Compute futures variance
    futures=yf.download(futures_ticker, start="2024-01-01")
    futures.dropna(inplace=True)
    futures.ffill(inplace=True)
    futures['Log Return']=np.log(futures['Adj Close']/futures['Adj Close'].shift(1))
    futures_var=futures['Log Return'].std()*np.sqrt(252)
    futures_price = futures['Adj Close'].iloc[-1]


    ### Compute correlation between two assets
    rho=share['Log Return'].corr(futures['Log Return'])

    ### Compute minimum variance hedge ratio
    h = rho*(share_var/futures_var)

    VA = share_price*QA
    VF = futures_price*QF

    ### Compute the number of contracts to hedge the futures and lock the price
    N=h*(VA/VF)
    return round(N)

In [31]:
cross_hedge(share_ticker='AAPL', futures_ticker='ES=F', QA=1000000, QF=10000)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


4

### Build a portfolio to hedge a futures

In [94]:
### The goal is to replicate the futures price in order to hedge the futures

prices = futures['Adj Close'].rename("Futures S&P 500")
tickers_list = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
def get_portfolio(tickers_list):
    df=pd.DataFrame()
    for ticker in tickers_list:
        share_prices = yf.download(ticker, start='2024-01-01', progress=False)['Adj Close'].rename(ticker)
        df=pd.concat([df, share_prices], axis=1)
    return df

In [75]:
def get_normalized_series(series):
    return series / series.iloc[0]

def get_portfolio_value(tickers_list, weights):
    portfolio=get_portfolio(tickers_list)
    value=pd.Series(0, index=portfolio.index)
    for i in range (len(tickers_list)):
        normalized_prices = get_normalized_series(portfolio.iloc[:, i])
        value+=weights[i]*normalized_prices
    return value

In [83]:
def replicate_portfolio (tickers_list, weights, futures_prices):
    futures_prices_normalized = get_normalized_series(futures_prices)
    portfolio_value = get_portfolio_value(tickers_list, weights)
    aligned_portfolio, aligned_futures = portfolio_value.align(futures_prices_normalized, join='inner')
    return abs(aligned_portfolio-aligned_futures)

In [95]:
def minimize_portfolio(tickers_list, futures_prices):
    """

    """
    n=len(tickers_list)
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n

    def objective(weights):
        replication_error = replicate_portfolio(tickers_list, weights, futures_prices)
        return replication_error.sum()

    results = minimize(objective, init_guess, method='SLSQP', options={'disp' : False}, bounds=bounds)
    return results.x

In [97]:
optimal_weights=minimize_portfolio(tickers_list, prices)

In [103]:
def get_real_portfolio_value(tickers_list, weights, prices):
    portfolio=get_portfolio(tickers_list)
    value=pd.Series(0, index=portfolio.index)
    for i in range (len(tickers_list)):
        value+=weights[i]*portfolio.iloc[:, i]*(prices.iloc[0]/portfolio.iloc[0, i])
    return value

In [105]:
get_real_portfolio_value(tickers_list, optimal_weights, prices)

2024-01-02    4676.717246
2024-01-03    4636.564118
2024-01-04    4541.784154
2024-01-05    4546.869702
2024-01-08    4662.359785
                 ...     
2024-10-24    5806.438173
2024-10-25    5842.409603
2024-10-28    5871.279827
2024-10-29    5920.562299
2024-10-30    5920.819557
Length: 210, dtype: float64

In [ ]:
### Final Solution

prices = futures['Adj Close'].rename("Futures S&P 500")
tickers_list = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']

df=pd.DataFrame()
for ticker in tickers_list:
    share_prices = yf.download(ticker, start='2024-01-01')['Adj Close'].rename(ticker)
    df=pd.concat([df, share_prices], axis=1)

def get_normalized_series(series):
    return series / series.iloc[0]

def get_portfolio_value(df, weights):
    value=pd.Series(0, index=df.index)
    for i in range (len(tickers_list)):
        normalized_prices = get_normalized_series(df.iloc[:, i])
        value+=weights[i]*normalized_prices
    return value

def replicate_portfolio (df, weights, futures_prices):
    futures_prices_normalized = get_normalized_series(futures_prices)
    portfolio_value = get_portfolio_value(df, weights)
    aligned_portfolio, aligned_futures = portfolio_value.align(futures_prices_normalized, join='inner')
    return abs(aligned_portfolio-aligned_futures)

def minimize_portfolio(df, futures_prices):
    """

    """
    n=len(tickers_list)
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n

    def objective(weights):
        replication_error = replicate_portfolio(df, weights, futures_prices)
        return replication_error.sum()

    results = minimize(objective, init_guess, method='SLSQP', options={'disp' : False}, bounds=bounds)
    return results.x

optimal_weights=minimize_portfolio(df, prices)

def get_real_portfolio_value(df, weights, prices):
    value=pd.Series(0, index=df.index)
    for i in range (len(df)):
        value+=weights[i]*portfolio.iloc[:, i]*(prices.iloc[0]/portfolio.iloc[0, i])
    return value